In [18]:
import xarray as xr
from dask.distributed import Client 


In [11]:
mesh = xr.open_dataset('/home/sallen/MEOPAR/grid/mesh_mask202108.nc')
# the mask is 1 where there is water, we want the opposite.  The meshmask has an extra dimension, hence the [0]
tmask = 1 - mesh.tmask[0]

volume = (1-tmask) * mesh.e3t_0[0] * mesh.e1t[0] * mesh.e2t[0]

e3t_1d = mesh.e3t_1d.rename({'z':'depth'}).isel(t=0)
e3t_1d.data

array([ 1.00000115,  1.00000501,  1.00001253,  1.00002718,  1.0000557 ,
        1.00011125,  1.00021946,  1.0004302 ,  1.00084067,  1.00164012,
        1.0031971 ,  1.00622914,  1.01213271,  1.02362358,  1.04597551,
        1.08940061,  1.17356428,  1.33592899,  1.64636781,  2.22990285,
        3.29248567,  5.11998508,  7.97451506, 11.8252972 , 16.10792044,
       19.95870258, 22.81323256, 24.64073198, 25.70331479, 26.28684983,
       26.59728865, 26.75965336, 26.84381704, 26.88724213, 26.90959407,
       26.92108493, 26.9269885 , 26.93002054, 26.93157752, 26.93237697])

In [14]:
chunk_sizes = {'time': 1, 'depth': 40, 'gridY': 898, 'gridX': 398}

In [15]:
fw = xr.open_dataset('/ocean/cdonaldson/research_runs/hindcast_auto/extractions/SalishSeaCast_river_trace_day_avg_allR_00_40_20120101_20121231.nc',chunks=chunk_sizes)

In [16]:
fw

<xarray.Dataset>
Dimensions:          (time: 366, depth: 40, gridY: 898, gridX: 398)
Coordinates:
  * time             (time) datetime64[ns] 2012-01-01T12:00:00 ... 2012-12-31...
  * depth            (depth) float32 0.5 1.5 2.5 3.5 ... 360.7 387.6 414.5 441.5
  * gridY            (gridY) int64 0 1 2 3 4 5 6 ... 891 892 893 894 895 896 897
  * gridX            (gridX) int64 0 1 2 3 4 5 6 ... 391 392 393 394 395 396 397
Data variables: (12/13)
    nooksack_river   (time, depth, gridY, gridX) float32 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    skagit_river     (time, depth, gridY, gridX) float32 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    snohomish_river  (time, depth, gridY, gridX) float32 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    nisqually_river  (time, depth, gridY, gridX) float32 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    elwha_river      (time, depth, gridY, gridX) float32 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    cowichan_river   (time, depth, gridY, gridX) float32 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    ...               ...
    puntledge_river  (time, depth, gridY, gridX) float32 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    salmon_river     (time, depth, gridY, gridX) float32 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    homathko_river   (time, depth, gridY, gridX) float32 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    squamish_river   (time, depth, gridY, gridX) float32 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    fraser_river     (time, depth, gridY, gridX) float32 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
    other_rivers     (time, depth, gridY, gridX) float32 dask.array<chunksize=(1, 40, 898, 398), meta=np.ndarray>
Attributes:
    name:         SalishSeaCast_river_trace_day_avg_allR_00_40_20120101_20121231
    description:  Day-averaged river dye extracted from SalishSeaCast v202111...
    history:      2024-03-25 18:21 -07:00: Generated by `reshapr extract /oce...
    Conventions:  CF-1.6

In [19]:
client=Client(processes=True, n_workers=4)  # 4 is a good choice for n_workers even on salish, if I was the only person logged on or asked for space, could go as high as 16 but... be conscious of this
# threads_per_worker = 1 if needed, netcdf is not threadsafe now so be realiable by specifying 1 (at expense of just some efficiency)

In [21]:
vol = fw.nooksack_river * e3t_1d

In [23]:
vol.compute()

: 